In [1]:
import requests
from io import StringIO
import pandas as pd

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [3]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [4]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [5]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [6]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    try:
        data = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    except Exception as e:
        print(f"problem with {url=} and {e=}")
        continue
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [7]:
len(set(sentences))

14505

In [8]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [9]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

/home/mat/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mat/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


(14504, 768)

In [10]:
import faiss

In [11]:
d = sentence_embeddings.shape[1]
d

768

In [12]:
index = faiss.IndexFlatL2(d)

In [13]:
index.is_trained

True

In [14]:
index.add(sentence_embeddings)

In [15]:
index.ntotal

14504

In [16]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [17]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[12386  3418  7636  9023]]
CPU times: user 20.4 ms, sys: 22 µs, total: 20.5 ms
Wall time: 15.4 ms


In [20]:
data['sentence_A'].iloc[I[0]]

KeyError: 'sentence_A'

In [23]:
data

,0,1,2
0,NaN,Small dog chews on a big stick.,a dog shews on a big stick.
1,NaN,A tennis player hitting the ball.,Two boys splashing in the surf.
2,NaN,a lone snowboarder in the middle of a snowy gust,A snowboarder is throwing up snow as he rides ...
3,4.4,A pair of dogs playing with a purple ball.,Two dogs play with purple football.
4,0.6,a bird lands in the water.,a boat floats in the water.
...,...,...,...
1495,NaN,A man doing a trick on a skateboard.,A man in mid air on a skateboard.
1496,NaN,A young girl in swim goggles does the backstro...,A girl does the backstroke in the pool.
1497,5.0,A deer jumps a fence.,A deer is jumping over a fence.
1498,1.0,A young girl dressed in a Minnie mouse outfit ...,a man wearing a white suit holding a newspaper...


In [24]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [25]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [26]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

In [27]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[12386  3418  7636  9023]]
CPU times: user 2.18 ms, sys: 0 ns, total: 2.18 ms
Wall time: 9.61 ms
